In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
db = sqlite3.connect("Monarchs1997.db")

In [3]:
df = pd.read_csv("1997 Season Summary.csv")

        #convert pandas to SQL table
df.to_sql("monarchs1997", db, if_exists="replace")

os.getcwd()

C:\Users\Arink\Anaconda3\lib\site-packages\pandas\core\generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


'C:\\Users\\Arink\\Desktop\\github\\CodeLou\\Monarchs\\Data'

In [4]:
schema = pd.read_sql('SELECT * FROM sqlite_master ORDER by name', db)
schema

,type,name,tbl_name,rootpage,sql
0,index,ix_monarchs1997_index,monarchs1997,3,"CREATE INDEX ""ix_monarchs1997_index""ON ""monarc..."
1,table,monarchs1997,monarchs1997,2,"CREATE TABLE ""monarchs1997"" (\n""index"" INTEGER..."


## Checking DataType

In [5]:
data_type = pd.read_sql('SELECT * FROM PRAGMA_TABLE_INFO("monarchs1997")', db)
data_type

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Tag No.,TEXT,0,None,0
2,2,Sex,TEXT,0,None,0
3,3,Tagger,TEXT,0,None,0
4,4,Where Tagged,TEXT,0,None,0
5,5,Date Tagged,TEXT,0,None,0
6,6,Date Recovered,TEXT,0,None,0
7,7,Where Recovered,TEXT,0,None,0
8,8,Observed or Reported by,TEXT,0,None,0
9,9,Interval,TEXT,0,None,0


### Drop tagger and reporter identification

In [6]:
df.drop(["Tagger", "Sex", "Observed or Reported by", "Interval"], axis = 1, inplace = True)
df

,Tag No.,Where Tagged,Date Tagged,Date Recovered,Where Recovered,Est. Distance
0,RN109,"Lane Township,PA",9/16/97,3//98,"El Rosario, Mexico",1995.0
1,NT910,"Fargo,ND",8/30/97,3//98,"El Rosario, Mexico",1909.0
2,PB278,"Milan,MI",9/5/97,3//98,"El Rosario, Mexico",1826.0
3,NH341,"Egan,MN",9/1/97,3//98,"El Rosario, Mexico",1788.0
4,SA533,"Cambrige,OH",9/12/97,2/1/98,"El Rosario, Mexico",1787.0
...,...,...,...,...,...,...
98,NB344,"Ames,IA",9/17/97,9/20/97,"Ames, IA",1.0
99,RO322,"Lawrence,KS",NaN,10/17/97,"Comstock, TX",NaN
100,DZ859,"University Park,PA",NaN,2/18/98,"Pensacola Beach, FL",NaN
101,071AB,"Shawnee,KS",NaN,2/16/98,"Victoria, TX",NaN


### Rename columns to match other years

In [7]:
df.rename(columns={'Date Tagged': 'Tag Date', 'Date Recovered': 'Report Date', 'Est. Distance': 'Miles'}, inplace=True)

### Split & rename combined columns in Tagged category

In [8]:
df2 = df['Where Tagged'].str.split(',', expand=True)
df2.columns = ['tag_city', 'Tag State']
#df2.loc[df2['Tag State'].str.contains(r'\s'), 'Tag State'].value_counts() #See if value contains spaces
df2['Tag State'] = df2['Tag State'].str.strip()
df2

,tag_city,Tag State
0,Lane Township,PA
1,Fargo,ND
2,Milan,MI
3,Egan,MN
4,Cambrige,OH
...,...,...
98,Ames,IA
99,Lawrence,KS
100,University Park,PA
101,Shawnee,KS


### Split & Rename combined columns in Reported category
### Removed white space in Report State column

In [9]:
df3 = df['Where Recovered'].str.split(',', expand=True)
df3.columns = ['report_city', 'Report State']
df3['Report State'] = df3['Report State'].str.strip()
df3.loc[df3['Report State'] == 'Mexico', 'Report State'] = 'Michoacán'
df3

,report_city,Report State
0,El Rosario,Michoacán
1,El Rosario,Michoacán
2,El Rosario,Michoacán
3,El Rosario,Michoacán
4,El Rosario,Michoacán
...,...,...
98,Ames,IA
99,Comstock,TX
100,Pensacola Beach,FL
101,Victoria,TX


### Create new column Report Country & clean up

In [10]:
add_country = df['Where Recovered'].str.extract("(Mexico)", expand=True)
add_country.columns = ['Report Country']
add_country.loc[add_country['Report Country'] == 'Mexico', 'Report Country'] = 'MEXICO'
add_country.loc[add_country['Report Country'].isnull(), 'Report Country'] = 'USA'
add_country

,Report Country
0,MEXICO
1,MEXICO
2,MEXICO
3,MEXICO
4,MEXICO
...,...
98,USA
99,USA
100,USA
101,USA


### Create new table with unneeded columns dropped

In [11]:
df4 = pd.concat([df, df2, df3, add_country], axis=1)
df4.drop(['Where Tagged', 'Where Recovered', 'report_city', 'tag_city'], axis=1, inplace=True)
df5 = df4[['Tag No.','Tag State', 'Tag Date', 'Report State', 'Report Country', 'Report Date', 'Miles']]
new_table = df5.copy()

In [12]:
new_table['Season']='1997'

### Final check for unwanted spaces

In [13]:
# new_table.loc[new_table['Report State'].str.contains(r'\s', na=False), 'Report State'].value_counts()
# new_table.loc[new_table['Tag State'].str.contains(r'\s', na=False), 'Tag State'].value_counts()
# new_table.loc[new_table['Tag No.'].str.contains(r'\s', na=False), 'Tag No.'].value_counts()
# new_table.loc[new_table['Report Country'].str.contains(r'\s', na=False), 'Report Country'].value_counts()

### Convert date columns to DateTime datatypes

In [14]:
new_table["Tag Date"]= pd.to_datetime(new_table["Tag Date"]) 
new_table["Report Date"]= pd.to_datetime(new_table["Report Date"]) 
new_table.dtypes

Tag No.                   object
Tag State                 object
Tag Date          datetime64[ns]
Report State              object
Report Country            object
Report Date       datetime64[ns]
Miles                    float64
Season                    object
dtype: object

In [15]:
del df2, df3, df4, df5, add_country

### Export cleaned data to csv & pkl

In [16]:
new_table.to_pickle("Monarchs1997.pkl")
new_table.to_csv("csv/Monarchs1997.csv")